In [1]:
# test
#input = "2333133121414131402"
with open('input.txt', 'r') as fl:
    input = fl.read().strip()

In [2]:
# get the length of the compressed disk
n = sum(int(input[i]) for i in range(0, len(input), 2))

In [3]:
# make a labeled input with IDs for ease of use 
# [(ID or -1 for missing, size)]
labeled_input = []
id = 0
for i, v in enumerate(input):
    if i % 2 == 0:
        labeled_input.append((id, int(v)))
        id += 1
    else:
        labeled_input.append((-1, int(v)))

In [4]:
# now fill the disk with data we know will not move
pos = 0
disk = [None]*n
for id, sz in labeled_input:
    if id == -1: # missing
        pos += sz
    else:
        for _j in range(sz):
            disk[pos] = id
            pos += 1
            if pos >= n: #filled it up
                break
    if pos >= n: #filled it up with empty at end
        break
        
# now work backwards to fill in the gaps
n_free = sum(1 for v in disk if v is None)
files = list(reversed(list(filter(lambda x: x[0] != -1, labeled_input))))

# tool to work backwards to get the next free pos
pos = 0
def get_next_free_pos(pos):
    v = True
    while v is not None:
        pos += 1
        v = disk[pos]
    return pos

i = 0
while n_free > 0:
    id, sz = files[i]
    # get the next position to fill
    for _j in range(sz):
        if n_free == 0:
            break
        pos = get_next_free_pos(pos)
        disk[pos] = id
        n_free -= 1
    i += 1
    
        

In [5]:
# get the checksum
print(sum(i*id for i, id in enumerate(disk)))

6288599492129


In [6]:
## part 2 ... it was fun to never build the full fragged disk, but I think we need to. 

In [7]:
# builded the fragged disk
disk = []
for id, sz in labeled_input:
    v = id if id!=-1 else None
    disk.extend([v]*sz)

# get a snapshot of the empty space
empty = []
start = None
for i, v in enumerate(disk):
    if v is None:
        if start is None:
            start = i
    else:
        if start is not None:
            empty.append((start, i))
            start = None
            
# ok, now we go through the files backwards and try to slot them in
files = {id: (disk.index(id), sz) for id, sz in labeled_input if id != -1}
for id in range(max(files.keys()), 0, -1):
    # get the current location and size
    pos, sz = files[id]
    # see if we can place the file left to right in the free space
    for i, (st, nd) in enumerate(empty):
        if nd-st >= sz and st<pos: # we can place it
            disk[st:(st+sz)] = [id]*sz
            disk[pos:(pos+sz)] = [None]*sz
            # now we need to update empty
            if nd-st == sz:
                empty.pop(i)
            else:
                empty[i] = (st+sz, nd)
            break
    

In [8]:
# get the checksum
print(sum(i*id for i, id in enumerate(disk) if id is not None))

6321896265143
